# 2.0

In [2]:
library('igraph')
library('Matrix')
library('stringr')


Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union



# question 18

In [ ]:
google_filepath = "google+/gplus"
file_names = list.files(path=google_filepath, pattern="circle")
num_personal_networks = 0
circle_file_over_two = character()
for( circle_file in file_names ){
    circle_path = paste("google+/gplus/", circle_file,sep="")
    circle_connect = file(circle_path, open="r")
    circle_content = readLines(circle_connect)
    close(circle_connect)
    if(length(circle_content)>2){
        num_personal_networks = num_personal_networks+1
        circle_file = str_replace_all(circle_file, fixed(".circles"),"")
        circle_file_over_two = c(circle_file_over_two, circle_file)
    }
}
print(paste("There are ", num_personal_networks," that have more than 2 circles", sep=""))


# question 19 and question 20

In [ ]:
# create graph file for 109327480479767108490, 115625564993990145546, 101373961279443806744 and extract community
file_ids = c("109327480479767108490", "115625564993990145546", "101373961279443806744")
for(id in circle_file_over_two){
    if(id %in% file_ids){
        graph_path = paste("google+/gplus/", id, ".edges", sep="")
        network = read.graph(graph_path, format = "ncol", directed=TRUE)
        network = add.vertices(network, nv = 1, name = id)
        ego_node = which(V(network)$name==id)
        ego_edges = c()
        for(vertex in 1:vcount(network)){
            if(vertex != ego_node)
                ego_edges = append(ego_edges, c(ego_node, vertex))
        }
        network = add_edges(network, ego_edges)
# plot in degree and out degree plot
        plot(degree.distribution(network, mode = "in"),main=paste("In Degree distribution of ", id, sep=""), xlab="In Degree",ylab="Frequency")
        plot(degree.distribution(network, mode = "out"),main=paste("Out Degree distribution of ", id, sep=""), xlab="Out Degree",ylab="Frequency")
# counting modularity score and plot community
        community = walktrap.community(network)
        print(paste("The modularity score for ", id, " is: ", modularity(community), sep = ""))
        plot(community, network, vertex.size = 5, vertex.label=NA, ,main=paste("Community graph of ", id, sep=""))
    }
}

## Question 21

homogeneity: the propotion of the difference of the entropy with every circles and the entropy with every circles given its community, and the entropy with every circles. In a nut shell, it measures the score that each community contains only people of a single circle.

completeness: the propotion of the difference of the entropy with every communities and the entropy with every communities given its circles, and the entropy with every communities. In a nut shell, it measures the score that all people of a given circle are assigned to the same community.

## Question 22

• C is the set of circles, C = {C1,C2,C3,···}

• K is the set of communities, K = {K1,K2,K3,···}

• ai is the number of people in circle Ci

• bi is the number of people in community Ki with circle information

• N is the total number of people with circle information

• Cji is the number of people belonging to community j and circle i

In [9]:
contigency_maxtrix = function(C, K){
   #construct zero matrix to calculate Cji
    contigency = matrix(nrow=length(C),ncol=length(K))
    contigency[is.na(contigency)] = 0
    # for each people in C, find its community and modify the contingency matrix
    for (i in 1:length(C)){
        for (people in C[[i]]){
            for (j in 1:length(K)){
                if (people %in% K[[j]]){
                    contigency[i,j] = contigency[i,j] + 1
                }
            }
        }
    }
    return (contigency)
}
    

In [4]:
homogeneity_score = function(C, K, N, contigency){
    H_C = 0
    H_CK = 0
    for (i in 1:length(C)){
        propotion = length(C[[i]]) / N
        # calculate H(C)
        H_C = H_C - (propotion * log2(propotion)) 
    }
    for(j in 1:length(K)){
        # calculate H(C|K)
        for (i in 1:length(C)){
            if (contigency[i,j] != 0){
                H_CK = H_CK - (contigency[i,j] / N) * (log2(contigency[i,j] / length(K[[j]])))
            }
        }
    }
    homogeneity = 1 - H_CK / H_C
    return (homogeneity)
}

In [5]:
completeness_score = function(C, K, N, contigency){
    H_K = 0
    H_KC = 0
    for(j in 1:length(K)){
        propotion = length(K[[j]]) / N
        H_K = H_K + (propotion * log2(propotion)) 
    }
    for(j in 1:length(K)){
        for (i in 1:length(C)){
            if (contigency[i,j] != 0){
                H_KC = H_KC + (contigency[i,j] / N) * log2(contigency[i,j] / length(C[[i]]))
            }
        }
    }
    completeness = 1 - H_KC / H_K
    return (completeness)
}

## For Node 109327480479767108490

In [10]:
id = c("109327480479767108490")
C = c()
class = c()
circle_path = paste("google+/gplus/", id, '.circles',sep="")
circle_connect = file(circle_path, open="r")
circle_content = readLines(circle_connect)
for (circle in circle_content){
    circle = unlist(strsplit(circle, '\t'))
    for (i in 2:length(circle)){
        class = append(class, circle[i])
    }
    C = append(C, list(class))
}
C = unique(C)

In [14]:
id = c("109327480479767108490")
C = c()
class = c()
circle_path = paste("google+/gplus/", id, '.circles',sep="")
circle_connect = file(circle_path, open="r")
circle_content = readLines(circle_connect)
for (circle in circle_content){
    circle = unlist(strsplit(circle, '\t'))
    for (i in 2:length(circle)){
        class = append(class, circle[i])
    }
    C = append(C, class)
}
C = unique(C)

In [15]:
id = c("109327480479767108490")
K = c()
graph_path = paste("google+/gplus/", id, ".edges", sep="")
network = read.graph(graph_path, format = "ncol", directed=TRUE)
network = add.vertices(network, nv = 1, name = id)
ego_node = which(V(network)$name==id)
ego_edges = c()
for(vertex in 1:vcount(network)){
    if(vertex != ego_node)
        ego_edges = append(ego_edges, c(ego_node, vertex))
}
network = add_edges(network, ego_edges)
community = walktrap.community(network)
for (i in 1:max(community$membership)){
    K = append(K, list(community[[i]]))
}

In [16]:
N = 0
for (i in 1:length(C)){
    N = N + length(C[[i]])
}
contigency = contigency_maxtrix(C, K)
print(N)
print(contigency)

[1] 764
       [,1] [,2] [,3] [,4]
  [1,]    1    0    0    0
  [2,]    1    0    0    0
  [3,]    1    0    0    0
  [4,]    1    0    0    0
  [5,]    1    0    0    0
  [6,]    1    0    0    0
  [7,]    1    0    0    0
  [8,]    1    0    0    0
  [9,]    1    0    0    0
 [10,]    0    0    1    0
 [11,]    1    0    0    0
 [12,]    1    0    0    0
 [13,]    1    0    0    0
 [14,]    1    0    0    0
 [15,]    1    0    0    0
 [16,]    1    0    0    0
 [17,]    1    0    0    0
 [18,]    1    0    0    0
 [19,]    1    0    0    0
 [20,]    1    0    0    0
 [21,]    1    0    0    0
 [22,]    1    0    0    0
 [23,]    1    0    0    0
 [24,]    1    0    0    0
 [25,]    0    0    1    0
 [26,]    1    0    0    0
 [27,]    1    0    0    0
 [28,]    1    0    0    0
 [29,]    0    0    1    0
 [30,]    1    0    0    0
 [31,]    1    0    0    0
 [32,]    1    0    0    0
 [33,]    1    0    0    0
 [34,]    1    0    0    0
 [35,]    1    0    0    0
 [36,]    0    0    

In [ ]:
homogeneity_score(C, K, N, contigency)

In [ ]:
completeness_score(C, K, N, contigency)

# 115625564993990145546

In [ ]:
id = c("115625564993990145546")
C = c()
class = c()
circle_path = paste("google+/gplus/", id, '.circles',sep="")
circle_connect = file(circle_path, open="r")
circle_content = readLines(circle_connect)
for (circle in circle_content){
    circle = unlist(strsplit(circle, '\t'))
    for (i in 2:length(circle)){
        class = append(class, circle[i])
    }
    C = append(C, list(class))
}

id = c("115625564993990145546")
K = c()
graph_path = paste("google+/gplus/", id, ".edges", sep="")
network = read.graph(graph_path, format = "ncol", directed=TRUE)
network = add.vertices(network, nv = 1, name = id)
ego_node = which(V(network)$name==id)
ego_edges = c()
for(vertex in 1:vcount(network)){
    if(vertex != ego_node)
        ego_edges = append(ego_edges, c(ego_node, vertex))
}
network = add_edges(network, ego_edges)
community = walktrap.community(network)
for (i in 1:max(community$membership)){
    K = append(K, list(community[[i]]))
}
N = 0
for (i in 1:length(C)){
    N = N + length(C[[i]])
}
contigency = contigency_maxtrix(C, K)
homogeneity_score(C, K, N, contigency)
completeness_score(C, K, N, contigency)

# 101373961279443806744

In [ ]:
id = c("101373961279443806744")
C = c()
class = c()
circle_path = paste("google+/gplus/", id, '.circles',sep="")
circle_connect = file(circle_path, open="r")
circle_content = readLines(circle_connect)
for (circle in circle_content){
    circle = unlist(strsplit(circle, '\t'))
    for (i in 2:length(circle)){
        class = append(class, circle[i])
    }
    C = append(C, list(class))
}

id = c("101373961279443806744")
K = c()
graph_path = paste("google+/gplus/", id, ".edges", sep="")
network = read.graph(graph_path, format = "ncol", directed=TRUE)
network = add.vertices(network, nv = 1, name = id)
ego_node = which(V(network)$name==id)
ego_edges = c()
for(vertex in 1:vcount(network)){
    if(vertex != ego_node)
        ego_edges = append(ego_edges, c(ego_node, vertex))
}
network = add_edges(network, ego_edges)
community = walktrap.community(network)
for (i in 1:max(community$membership)){
    K = append(K, list(community[[i]]))
}
N = 0
for (i in 1:length(C)){
    N = N + length(C[[i]])
}
contigency = contigency_maxtrix(C, K)
homogeneity_score(C, K, N, contigency)
completeness_score(C, K, N, contigency)